# Simple Application of trained ROOT TMVA classifiers

Here we take the results of RunTrainSignalID and add the classifier response to a tree or evaluate it in a loop.

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


### Import chanser_mva stuff

In [2]:
import ROOT
from ROOT import TMVA, TFile, TH1F, TCanvas
TMVA.Tools.Instance()
ROOT.TMVA.PyMethodBase.PyInitialize()
ROOT.gROOT.ProcessLine(".x $CHANSER/macros/Load.C")
ROOT.gSystem.Load("$CHANSER/lib/libchansermva.so")
from ROOT import chanser
from ROOT.chanser import mva
from ROOT.chanser.mva import ResultInterface
from ROOT.chanser.mva import ResultByFloatTree
from ROOT.chanser.mva import ResultByTree
from ROOT.chanser.mva import Meths

Welcome to JupyROOT 6.20/02


jsroot this will work with root --notebook, but not jupyter-notebook

In [3]:
%jsroot on

### Get the tutorial data file. 
This is the same as is used in the ROOT TMVA classification tutorial

In [4]:
if ROOT.gSystem.AccessPathName( "tmva_class_example.root" ) != 0: 
    ROOT.gSystem.Exec( "wget https://root.cern.ch/files/tmva_class_example.root")
    
input = TFile.Open( "tmva_class_example.root" )

# Get the signal tree for applying the trained respense to
aTree      = input.Get( "TreeS" )


### Create the results object. 
It needs the top level directory that contains the training Training.root file as the first argument.

The second argument is the classifier name you want to use here. During training you may have used many classifiers but here we only use one at a time. The last argument is the tree you want to evaluate the classifier for, **Note this does not have to be the same as any of the training trees**. In this part we are just going to get the value while looping over the tree events.

We use the interface ResultByFloatTree which requires all the branches in the tree are already floats. If this is not the case you can use ResultByTree as shown below

Here we use the classifier we called MLP, which was a multi-layer perceptron from the training example

In [5]:
classif = ResultByFloatTree("TMVAClassificationTut","MLP",aTree);

create data set info Default
create data set info Default
create data set info 
var1
var2
var3
var4
                         : Booking "MLP" of type "MLP" from TMVAClassificationTut/weights/TMVAClassificationTut_MLP.weights.xml.
                         : Reading weight file: TMVAClassificationTut/weights/TMVAClassificationTut_MLP.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
<HEADER> MLP                      : Building Network. 
                         : Initializing weights
                         : Booked classifier "MLP" of type: "MLP"


### Event Loop
Loop over all events in the tree and fill a histogram

In [6]:
hresponse    = TH1F( 'response', 'Response of my trained classifier', 100, -1,1 )
hresponse.SetFillColor( 48 )

for event in aTree :
    hresponse.Fill( classif.Eval())


In [7]:
c1 = TCanvas( 'c1', 'response canvas', 200, 10, 700, 500 )
hresponse.Draw()
c1.Draw()

### Adding the classifier response to a tree

You can directly add a response branch to a tree. In this case we use ResultByTree in case there are non float branches in the tree. We also have the option of just adding the branch to the tree in memory without writing it to file, which is done here, or just copying the tree and writing to a new file of name classifier_name.root e.g. MLP.root . This is steered by the last boolean argument 0 => just add branch here in memory

In [8]:
classif2= ResultByTree("TMVAClassificationTut","MLP",aTree,0)
classif2.AddToTree()

create data set info Default
create data set info Default
create data set info 
 ResultInterface::SetBranchAddresses found branches : 
   double 0
   float 4
   long 0
   int 0
   total vars in reader are 4
                         : Booking "MLP" of type "MLP" from TMVAClassificationTut/weights/TMVAClassificationTut_MLP.weights.xml.
                         : Reading weight file: TMVAClassificationTut/weights/TMVAClassificationTut_MLP.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
<HEADER> MLP                      : Building Network. 
                         : Initializing weights
                         : Booked classifier "MLP" of type: "MLP"
 ResultInterface::AddToTree() 0x563461606150 0x563460bc2470
 ResultInterface::AddClassificationToTree()


You can now draw the reponse from the tree

In [9]:
c2 = TCanvas( 'c2', 'response from tree', 200, 10, 700, 500 )
aTree.Draw("var1:MLP")
c2.Draw()